<a href="https://colab.research.google.com/github/Auxilus08/Compiler-Design/blob/main/Cd_practical3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parser Construction

In [22]:
import sys
grammer = {}
non_terminals = set()
first_sets = {}


In [23]:
def parse_grammer(input_lines):
  grammer.clear()
  non_terminals.clear()

  for line in input_lines:
    line = line.strip()
    if not line or line.startswith('#'):
      continue
    head, _, str_productions = line.partition(':')
    head = head.strip()

    if not head.isupper():
      print(f" Non terminal {head} is not uppercase.")

    non_terminals.add(head)

    if head not in grammer:
      grammer[head] = []

    productions = str_productions.split('|')
    for production in productions:
      production = production.strip()
      grammer[head].append(production.split()) # Split production into symbols

In [24]:
def calculate_first_for_sequence(sequence):
  first = set()

  if not sequence or (len(sequence) == 1 and sequence[0] == 'Epsilon'): # Handle single 'Epsilon'
    return {'Epsilon'}

  can_produce_epsilon = True

  for symbol in sequence:
    if symbol not in non_terminals:
      first.add(symbol)
      can_produce_epsilon = False
      break

    first_of_symbol = first_sets.get(symbol, set()) # Use .get() with empty set default
    first.update(first_of_symbol - {'Epsilon'})

    if 'Epsilon' not in first_of_symbol:
      can_produce_epsilon = False
      break

  if can_produce_epsilon:
    first.add('Epsilon')

  return first

In [25]:
def compute_first_set():

  for nt in non_terminals:
    first_sets[nt] = set()

  while True:
    changed = False

    for nt in non_terminals:
      original_size = len(first_sets[nt])

      for production in grammer.get(nt, []):
        first_of_rhs = calculate_first_for_sequence(production)
        first_sets[nt].update(first_of_rhs)

      if len(first_sets[nt]) != original_size:
        changed = True

    if not changed:
      break

In [26]:
def main():
  print("Enter the Grammer Rules: ")
  print("use 'Epsilon'. press Enter on an Empty line when you are done")
  print("-" * 20)

  input_lines = []
  while True:
    try:
      line = input()
      if not line:
        break
      input_lines.append(line)

    except EOFError:
      break

  if not input_lines:
    print("No grammer rules for Entered. Exiting")
    return

  try:
    parse_grammer(input_lines)
    compute_first_set()

    sorted_nts = sorted(list(non_terminals))
    for nt in sorted_nts:
            terminals_str = ", ".join(sorted(list(first_sets.get(nt, set()))))
            print(f"FIRST({nt}) = {{ {terminals_str} }}")

  except Exception as e:
        print(f"\nAn error occurred: {e}", file=sys.stderr)



In [28]:
if __name__ == "__main__":
    main()

Enter the Grammer Rules: 
use 'Epsilon'. press Enter on an Empty line when you are done
--------------------
A : S B | B
B : b | d
S : a | B c | Epsilon

FIRST(A) = { a, b, d }
FIRST(B) = { b, d }
FIRST(S) = { Epsilon, a, b, d }
